In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("darkgrid")
import datetime

from kiblib.utils.db import DbConn

In [2]:
db_conn = DbConn().create_engine()

In [142]:
query = """
SELECT * FROM statdb.stat_issues
WHERE location = 'MED0C' AND DATE(issuedate) >= CURDATE() - INTERVAL 1 YEAR
"""
prets = pd.read_sql(query, con=db_conn)
prets = prets[prets["ccode"].str[0:5] != 'AAPBD']
prets = prets[~prets["ccode"].isin(['P17', 'ALTFAZZ'])]
prets['fr'] = np.nan
prets.loc[prets['itemcallnumber'].str.match(r'^ROUB', case=False, na=False), 'fr'] = 'ROUBAIX'
prets.loc[prets['itemcallnumber'].str.match(r'^FR(\/|\s)BD', case=False, na=False), 'fr'] = 'FR BD'
prets.loc[prets['itemcallnumber'].str.match(r'^FR(\/|\s)PICARD', case=False, na=False), 'fr'] = 'FR PICARD'
prets.loc[prets['itemcallnumber'].str.match(r'^FR(\/|\s)RP', case=False, na=False), 'fr'] = 'FR RP'
prets.loc[prets['itemcallnumber'].str.match(r'^FR(\/|\s)R(\/|\s)', case=False, na=False), 'fr'] = 'FR R'
prets.loc[prets['itemcallnumber'].str.match(r'^FR(\/|\s)\d', case=False, na=False), 'fr'] = 'FR 004-956'

In [144]:
prets_fr = prets.groupby(['fr']).agg({"issuedate": 'count',
                                      "itemnumber": pd.Series.nunique,
                                      "borrowernumber": pd.Series.nunique}).reset_index()
prets_fr = prets_fr.rename(columns={'issuedate': 'prêts', 'itemnumber':'exemplaires_uniques', 'borrowernumber': 'emprunteurs'})
prets_fr

,fr,prêts,exemplaires_uniques,emprunteurs
0,FR 004-956,742,436,259
1,FR BD,414,213,161
2,FR PICARD,2,1,2
3,FR R,573,269,259
4,FR RP,505,252,134
5,ROUBAIX,209,121,82


In [136]:
query = """
SELECT itemnumber, ccode, itemcallnumber FROM koha_prod.items
WHERE location = 'MED0C' AND notforloan = 0
"""
items = pd.read_sql(query, con=db_conn)

query = """
SELECT itemnumber FROM koha_prod.items
WHERE location = 'MED0C' AND notforloan = 0
AND itemnumber IN (SELECT itemnumber FROM koha_prod.issues)
"""
items_inissues = pd.read_sql(query, con=db_conn)
items_inissues['inissue'] = True

items = items.merge(items_inissues, on='itemnumber', how='left')
items['inissue'] = items['inissue'].fillna(False)

In [137]:
items = items[items["ccode"].str[0:5] != 'AAPBD']
items = items[~items["ccode"].isin(['P17', 'ALTFAZZ'])]
items['fr'] = np.nan
items.loc[items['itemcallnumber'].str.match(r'^ROUB', case=False, na=False), 'fr'] = 'ROUBAIX'
items.loc[items['itemcallnumber'].str.match(r'^FR(\/|\s)BD', case=False, na=False), 'fr'] = 'FR BD'
items.loc[items['itemcallnumber'].str.match(r'^FR(\/|\s)PICARD', case=False, na=False), 'fr'] = 'FR PICARD'
items.loc[items['itemcallnumber'].str.match(r'^FR(\/|\s)RP', case=False, na=False), 'fr'] = 'FR RP'
items.loc[items['itemcallnumber'].str.match(r'^FR(\/|\s)R(\/|\s)', case=False, na=False), 'fr'] = 'FR R'
items.loc[items['itemcallnumber'].str.match(r'^FR(\/|\s)\d', case=False, na=False), 'fr'] = 'FR 004-956'

In [138]:
items_fr = items.groupby(['fr']).agg({"itemnumber": 'count'}).reset_index()
items_fr = items_fr.rename(columns={'itemnumber': 'exemplaires'})
items_fr

,fr,exemplaires
0,FR 004-956,1721
1,FR BD,370
2,FR PICARD,35
3,FR R,603
4,FR RP,431
5,ROUBAIX,335


In [145]:
items_fr_inissue = items[items['inissue']  == True].groupby(['fr']).agg({"itemnumber": 'count'}).reset_index()
items_fr_inissue = items_fr_inissue.rename(columns={'itemnumber': 'exemplaires_sorties'})
items_fr_inissue

,fr,exemplaires_sorties
0,FR 004-956,50
1,FR BD,30
2,FR R,37
3,FR RP,40
4,ROUBAIX,15


In [148]:
fond_reg = items_fr.merge(prets_fr, on='fr')
fond_reg = fond_reg.merge(items_fr_inissue, on='fr')
fond_reg.columns

Index(['fr', 'exemplaires', 'prêts', 'exemplaires_uniques', 'emprunteurs',
       'exemplaires_sorties'],
      dtype='object')

In [150]:
fond_reg['tx_rotation'] = round(fond_reg['prêts'] / fond_reg['exemplaires'], 2)
fond_reg['tx_actif'] = round(fond_reg['exemplaires_uniques'] / fond_reg['exemplaires'], 2)
fond_reg['tx_sortie'] = round(fond_reg['exemplaires_sorties'] / fond_reg['exemplaires'], 2)
fond_reg[['fr', 'exemplaires', 'prêts', 'emprunteurs', 'tx_rotation', 'tx_actif', 'tx_sortie']]

,fr,exemplaires,prêts,emprunteurs,tx_rotation,tx_actif,tx_sortie
0,FR 004-956,1721,742,259,0.43,0.25,0.03
1,FR BD,370,414,161,1.12,0.58,0.08
2,FR R,603,573,259,0.95,0.45,0.06
3,FR RP,431,505,134,1.17,0.58,0.09
4,ROUBAIX,335,209,82,0.62,0.36,0.04


In [3]:
query = """
SELECT * FROM statdb.stat_issues
WHERE location = 'MED0C' AND DATE(issuedate) >= CURDATE() - INTERVAL 1 YEAR
"""
prets = pd.read_sql(query, con=db_conn)
prets = prets[prets["ccode"].isin(['PRRMEZZ', 'PRRFIZZ'])]
prets['fr'] = prets["ccode"]

In [4]:
prets_fr = prets.groupby(['fr']).agg({"issuedate": 'count',
                                      "itemnumber": pd.Series.nunique,
                                      "borrowernumber": pd.Series.nunique}).reset_index()
prets_fr = prets_fr.rename(columns={'issuedate': 'prêts', 'itemnumber':'exemplaires_uniques', 'borrowernumber': 'emprunteurs'})
prets_fr

,fr,prêts,exemplaires_uniques,emprunteurs
0,PRRFIZZ,364,136,100
1,PRRMEZZ,143,96,49


In [5]:
query = """
SELECT itemnumber, ccode, itemcallnumber FROM koha_prod.items
WHERE location = 'MED0C' AND notforloan = 0
"""
items = pd.read_sql(query, con=db_conn)

query = """
SELECT itemnumber FROM koha_prod.items
WHERE location = 'MED0C' AND notforloan = 0
AND itemnumber IN (SELECT itemnumber FROM koha_prod.issues)
"""
items_inissues = pd.read_sql(query, con=db_conn)
items_inissues['inissue'] = True

items = items.merge(items_inissues, on='itemnumber', how='left')
items['inissue'] = items['inissue'].fillna(False)

In [6]:
items = items[items["ccode"].isin(['PRRMEZZ', 'PRRFIZZ'])]
items['fr'] = items["ccode"]

In [7]:
items_fr = items.groupby(['fr']).agg({"itemnumber": 'count'}).reset_index()
items_fr = items_fr.rename(columns={'itemnumber': 'exemplaires'})
items_fr

,fr,exemplaires
0,PRRFIZZ,224
1,PRRMEZZ,527


In [8]:
items_fr_inissue = items[items['inissue']  == True].groupby(['fr']).agg({"itemnumber": 'count'}).reset_index()
items_fr_inissue = items_fr_inissue.rename(columns={'itemnumber': 'exemplaires_sorties'})
items_fr_inissue

,fr,exemplaires_sorties
0,PRRFIZZ,17
1,PRRMEZZ,6


In [9]:
fond_reg = items_fr.merge(prets_fr, on='fr')
fond_reg = fond_reg.merge(items_fr_inissue, on='fr')
fond_reg.columns

Index(['fr', 'exemplaires', 'prêts', 'exemplaires_uniques', 'emprunteurs',
       'exemplaires_sorties'],
      dtype='object')

In [10]:
fond_reg['tx_rotation'] = round(fond_reg['prêts'] / fond_reg['exemplaires'], 2)
fond_reg['tx_actif'] = round(fond_reg['exemplaires_uniques'] / fond_reg['exemplaires'], 2)
fond_reg['tx_sortie'] = round(fond_reg['exemplaires_sorties'] / fond_reg['exemplaires'], 2)
fond_reg[['fr', 'exemplaires', 'prêts', 'emprunteurs', 'tx_rotation', 'tx_actif', 'tx_sortie']]

,fr,exemplaires,prêts,emprunteurs,tx_rotation,tx_actif,tx_sortie
0,PRRFIZZ,224,364,100,1.62,0.61,0.08
1,PRRMEZZ,527,143,49,0.27,0.18,0.01
